In [ ]:

# Example:
# Dummy data for demonstration:
text1 = Text("T1", ((50, 10), (70, 30)), 0.95) # Center (60, 20)
text2 = Text("LabelP", ((90, 90), (150, 110)), 0.9) # Center (120, 100)
text3 = Text("Far away", ((300,300), (350,310)), 0.9) # Center (325, 305)

all_texts = [text1, text2, text3]

# A place centered at (100,100) with radius 20
place1_contour = np.array([[[80,100]], [[100,80]], [[120,100]], [[100,120]]], dtype=np.int32) # Approx
place1 = Place.from_contour(place1_contour) # Center approx (100,100), radius approx 20

# A transition (rectangle)
# Points for a rectangle approx from (40,40) to (80,60)
transition1_contour = np.array([[[40,40]], [[80,40]], [[80,60]], [[40,60]]], dtype=np.int32)
transition1 = Transition.from_contour(transition1_contour)

# An arc (line segment) from (0,0) to (50,0)
arc1_pt_start = Point(0,0)
arc1_pt_end = Point(50,0)
arc1 = Arc(source=None, target=None, start_point=arc1_pt_start, end_point=arc1_pt_end, points=[arc1_pt_start, arc1_pt_end])

all_places = [place1]
all_transitions = [transition1]
all_arcs = [arc1]

# Set a threshold
LINKING_THRESHOLD = 25.0 

link_text_to_elements(
    all_texts,
    all_places,
    all_transitions,
    all_arcs,
    LINKING_THRESHOLD
)

# Check associations
print("--- Associated Text ---")
for p in all_places:
    if p.text: print(f"{p} has text: {[t.value for t in p.text]}")
for t in all_transitions:
    if t.text: print(f"{t} has text: {[txt.value for txt in t.text]}") # changed t.text to txt.value
for a in all_arcs:
    if a.text: print(f"{a} has text: {[t.value for t in a.text]}")


In [3]:
CONFIG_PATH = 'config.yaml'
config = {}
with open(CONFIG_PATH, 'r') as f:
        config = yaml.safe_load(f)

In [5]:
INPUT_IMAGE_PATH = 'data/local/mid_petri_2.png'
preprocessed_img, img_color_resized, img_gray_resized = load_and_preprocess_image(INPUT_IMAGE_PATH, config)

Image dimensions (1057x619) are below threshold (800px). Upscaling by 2x.
New image dimensions: 2114x1238


In [25]:

predictor = ocr_predictor(
    det_arch='db_resnet50', 
    reco_arch='crnn_vgg16_bn', 
    pretrained=True,
    # Pass other params from config if needed, e.g.:
    assume_straight_pages= True
)
predictor.det_predictor.model.postprocessor.bin_thresh = 0.3
predictor.det_predictor.model.postprocessor.box_thresh =0.1

print("Running text detection...")
out = predictor([img_color_resized]) 


Running text detection...


In [13]:
out.pages[0].blocks[0].lines

[Line(
   (words): [Word(value='T1', confidence=1.0)]
 ),
 Line(
   (words): [
     Word(value='measure.storage.', confidence=0.62),
     Word(value='size', confidence=0.99),
   ]
 ),
 Line(
   (words): [Word(value='retur.to_storage', confidence=0.53)]
 ),
 Line(
   (words): [Word(value='canceLshop.order', confidence=0.6)]
 ),
 Line(
   (words): [Word(value='P1', confidence=1.0)]
 ),
 Line(
   (words): [Word(value='target_amount', confidence=0.97)]
 ),
 Line(
   (words): [Word(value='actual_size', confidence=0.96)]
 ),
 Line(
   (words): [Word(value='szetorders-unsatismed', confidence=0.32)]
 ),
 Line(
   (words): [Word(value='72', confidence=1.0)]
 ),
 Line(
   (words): [Word(value='19', confidence=1.0)]
 ),
 Line(
   (words): [Word(value='1', confidence=1.0)]
 ),
 Line(
   (words): [Word(value='0', confidence=0.99)]
 ),
 Line(
   (words): [Word(value='0', confidence=0.98)]
 ),
 Line(
   (words): [Word(value='P1', confidence=1.0)]
 ),
 Line(
   (words): [Word(value='t=4.0', confidence

In [28]:
h, w = img_gray_resized.shape
# mask = np.zeros((h, w), dtype=np.uint8)
mask = img_gray_resized.copy()

# Extract bounding boxes
# Adjusting based on typical doctr output structure (may need verification)
for block in out.pages[0].blocks:
    for line in block.lines:
        for word in line.words:
            # print(word.geometry)
            # Geometry is ((xmin, ymin), (xmax, ymax)) relative to image dims
            # Geometry is ((xmin, ymin), (xmax, ymax)) relative to image dims
            box = word.geometry
            rel_xmin, rel_ymin = map(float, box[0])
            rel_xmax, rel_ymax = map(float, box[1])

            # Convert relative coordinates to absolute pixel coordinates
            abs_xmin = int(round(rel_xmin * w))
            abs_ymin = int(round(rel_ymin * h))
            abs_xmax = int(round(rel_xmax * w))
            abs_ymax = int(round(rel_ymax * h))

            # Store as a list of points [ [x1,y1], [x2,y2], [x3,y3], [x4,y4] ]
            # Similar to your original _to_absolute function's output format
            points = np.array([
                [abs_xmin, abs_ymin], # Top-left
                [abs_xmax, abs_ymin], # Top-right
                [abs_xmax, abs_ymax], # Bottom-right
                [abs_xmin, abs_ymax]  # Bottom-left
            ], dtype=np.int32)
            print(points)
            # # Draw filled rectangle on mask
            cv2.rectangle(mask, (abs_xmin, abs_ymin), (abs_xmax, abs_ymax), (255), thickness=cv2.FILLED)

print("Created text mask.")

[[557  18]
 [590  18]
 [590  45]
 [557  45]]
[[ 809   16]
 [1026   16]
 [1026   47]
 [ 809   47]]
[[1012   18]
 [1061   18]
 [1061   45]
 [1012   45]]
[[1210   20]
 [1408   20]
 [1408   51]
 [1210   51]]
[[1877    6]
 [2097    6]
 [2097   43]
 [1877   43]]
[[407  41]
 [434  41]
 [434  62]
 [407  62]]
[[636  33]
 [803  33]
 [803  64]
 [636  64]]
[[1078   33]
 [1204   33]
 [1204   64]
 [1078   64]]
[[1429   31]
 [1685   31]
 [1685   60]
 [1429   60]]
[[648  70]
 [687  70]
 [687 109]
 [648 109]]
[[1862   66]
 [1895   66]
 [1895   93]
 [1862   93]]
[[415 105]
 [431 105]
 [431 121]
 [415 121]]
[[751  97]
 [770  97]
 [770 119]
 [751 119]]
[[1639   97]
 [1658   97]
 [1658  117]
 [1639  117]]
[[1875  121]
 [1908  121]
 [1908  150]
 [1875  150]]
[[545 171]
 [609 171]
 [609 202]
 [545 202]]
[[ 970  169]
 [1038  169]
 [1038  200]
 [ 970  200]]
[[1344  175]
 [1404  175]
 [1404  200]
 [1344  200]]
[[1887  188]
 [1899  188]
 [1899  204]
 [1887  204]]
[[2011  154]
 [2079  154]
 [2079  196]
 [2011  19

In [29]:
Image.fromarray(mask).show()

In [23]:
img_gray_inpainted = cv2.inpaint(img_gray_resized, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
Image.fromarray(img_gray_inpainted).show()

In [ ]:

# print(f"Text detection complete. Found {len(out.pages[0].blocks[0].lines)} blocks.")

# # --- Create Mask from Detected Text ---
# h, w = img_gray_resized.shape
# mask = np.zeros((h, w), dtype=np.uint8)

# # Extract bounding boxes
# # Adjusting based on typical doctr output structure (may need verification)
# for block in out.pages[0].blocks:
#     for line in block.lines:
#         for word in line.words:
#             # Geometry is ((xmin, ymin), (xmax, ymax)) relative to image dims
#             box = word.geometry
#             xmin, ymin = map(int, box[0])
#             xmax, ymax = map(int, box[1])
#             # Draw filled rectangle on mask
#             cv2.rectangle(mask, (xmin, ymin), (xmax, ymax), (255), thickness=cv2.FILLED)

# print("Created text mask.")

# # --- Inpaint Grayscale Image ---
# print("Inpainting grayscale image to remove text...")
# # Use Telea inpainting method as in the notebook
# img_gray_inpainted = cv2.inpaint(img_gray_resized, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
# print("Inpainting complete.")


### Else

In [ ]:
# ### draw entry points
# img_draw = img_color_resized.copy()
# for line in processed_lines:
#     cv2.line(img_draw, (line.point1.x, line.point1.y), (line.point2.x, line.point2.y), (np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256)), 2)

# for point in entry_points:
#     cv2.circle(img_draw, (int(point.x), int(point.y)), 5, (0, 255, 0), -1)

# Image.fromarray(img_draw).show()

In [ ]:
# img_draw = img_color_resized.copy()
# for node_copy in transitions:
#     ### draw the node copy
#     cv2.circle(img_draw, (node_copy.center.x, node_copy.center.y), 5, (0, 0, 255), -1)

#     expanded_height = node_copy.height * 1.4
#     expanded_width = node_copy.width * 3
    
    
#     expanded_bbox_contour = cv2.boxPoints(((node_copy.center.x, node_copy.center.y),
#                                             (expanded_height, expanded_width), node_copy.angle))
    
#     cv2.drawContours(img_draw, [expanded_bbox_contour.astype(np.int32)], -1, (0, 255, 0), 2)
    
# Image.fromarray(img_draw).show()

In [ ]:
# node_centers = [node.center for node in [*places, *transitions]]
# proximity_threshold = 1.5

# for line in lines:
#     for node_center in node_centers:
#         node = node_center.part_of
#         if isinstance(node, Place):
#             for line_point in [line.point1, line.point2]:
#                 if line_point.get_distance_between_points(node_center) < proximity_threshold*node.radius:
#                     line_point.proximity_node = node

#         elif isinstance(node, Transition):
#             for line_point in [line.point1, line.point2]:
#                 ### check if the line_point is inside the rectangle, which could be rotated
#                 expanded_height = node.height * proximity_threshold
#                 expanded_width = node.width * proximity_threshold
#                 bbox = cv2.boxPoints(((node_center.x, node_center.y), (expanded_width, expanded_height), node.angle))
#                 if cv2.pointPolygonTest(bbox, (line_point.x, line_point.y), False) >= 0:
#                     line_point.proximity_node = node

#         else: 
#             raise ValueError("Unknown node type")      



In [ ]:
# img_empty_nodes_filled = None
# nodes_mask = None
# detected_circles = []
# detected_rectangles = []
# img_no_shapes = None

# if img_no_text is not None:
#     try:
#         # print("\nFilling empty nodes...")
#         # img_empty_nodes_filled = fill_empty_nodes(img_no_text, config) # Pass img_no_text here
#         # Image.fromarray(img_empty_nodes_filled).show(title="After Filling Empty Nodes")

        

#         # print("\nIsolating nodes mask...")
#         # nodes_mask = get_nodes_mask(img_empty_nodes_filled, config) 
#         # Image.fromarray(nodes_mask).show(title="Isolated Nodes Mask")

#         # print("\nDetecting shapes (Places/Transitions)...")
#         # # Store the results for later use
#         # detected_circles, detected_rectangles = detect_shapes(nodes_mask, config)
#         # print(f"Detected {len(detected_circles)} potential Places (circles).")
#         # print(f"Detected {len(detected_rectangles)} potential Transitions (rectangles).")

#         # # Optional: Visualize detected shapes on the color image (create a copy)
#         # img_shapes_viz = img_color_resized.copy() if img_color_resized is not None else None
#         # if img_shapes_viz is not None:
#         #     for (x,y,radius) in detected_circles:
#         #         cv2.circle(img_shapes_viz, (x,y), radius, (0,255,0), 2) # Green for circles
#         #     for rect in detected_rectangles: # Assuming lir rectangles
#         #         # Use lir helpers pt1 and pt2 to get diagonal corners for cv2.rectangle
#         #         pt1 = lir.pt1(rect) 
#         #         pt2 = lir.pt2(rect)
#         #         cv2.rectangle(img_shapes_viz, pt1, pt2, (0,0,255), 2) # Red for rectangles
#         #     Image.fromarray(cv2.cvtColor(img_shapes_viz, cv2.COLOR_BGR2RGB)).show(title="Detected Shapes")


#         # print("\nRemoving detected nodes for connection processing...")
#         # # Use the image *before* empty node filling but *after* text removal for removing shapes
#         # img_no_shapes = remove_nodes(img_no_text, detected_circles, detected_rectangles, config)
#         # Image.fromarray(img_no_shapes).show(title="Image with Nodes Removed")

#     except Exception as e:
#         print(f"An error occurred during shape detection/removal: {e}")
#         # Ensure subsequent steps know these might be invalid
#         detected_circles = []
#         detected_rectangles = []
#         img_no_shapes = None
# else:
#     print("\nSkipping shape detection because text removal failed or was skipped.")

# # Old direct calls - replaced by the block above
# # img_empty_nodes_filled = fill_empty_nodes(preprocessed_img)
# # Image.fromarray(img_empty_nodes_filled).show()

# # nodes_mask = get_nodes_mask(img_empty_nodes_filled) 
# # Image.fromarray(nodes_mask).show()

# # circles, rectangles = detect_shapes(nodes_mask)
# # img_no_shapes = remove_nodes(preprocessed_img, circles, rectangles)
# # Image.fromarray(img_no_shapes).show()

# # %% 
# # Results from this section to be used later:
# # - detected_circles: List of (x, y, radius) tuples for Places
# # - detected_rectangles: List of lir rectangle objects for Transitions
# # - img_no_shapes: Image ready for connection processing


# # %% [markdown]
# # ## 6. Connection Processing (Arcs) (To be added)

# # %% 
# # Placeholder for connection processing logic
# # print("\nPlaceholder for Connection Processing module...") # Removed placeholder print


# # %% [markdown]
# # ## 7. Petri Net Construction (To be added)

# # %% 
# # Placeholder for Petri Net construction logic
# # print("\nPlaceholder for Petri Net Construction module...") # Removed placeholder print


# # %% [markdown]
# # ## 8. Output/Export (To be added)

# # %% 
# # Placeholder for output/export logic
# # print("\nPlaceholder for Output/Export module...") # Removed placeholder print


In [ ]:
# ### drawn node contours
# img_drawn = cv2.cvtColor(img_no_shapes, cv2.COLOR_GRAY2BGR) 
# for contour in detected_circles + detected_rectangles:
#     cv2.polylines(img_drawn, [contour], True, (0, 255, 0), 2)
# Image.fromarray(img_drawn).show(title="Detected Shapes")

# ### draw min area rectangles
# img_drawn = img_color_resized.copy()
# for i, rect in enumerate(detected_rectangles):
#     min_contour = cv2.minAreaRect(rect)
#     box = cv2.boxPoints(min_contour)
#     # print(f"box points {i}:", box)
#     cv2.drawContours(img_drawn, [box.astype(int)], -1, (0, 255, 0), 2)

# Image.fromarray(img_drawn).show(title="Detected Rectangles")

# cv2.minAreaRect(rectangles[0])[0], cv2.minAreaRect(rectangles[0])[1], cv2.minAreaRect(rectangles[0])[2]

In [ ]:
# t1 = transitions[0]

# img_drawn = img_color_resized.copy()
# cv2.drawContours(img_drawn, [t1.box_points.astype(int)], -1, (0, 255, 0), 2)
# expanded_height = t1.height * 1.5
# expanded_width = t1.width * 3


# expanded_bbox_contour = cv2.boxPoints(((t1.center.x, t1.center.y),
#                                         (expanded_height, expanded_width), t1.angle))

# cv2.drawContours(img_drawn, [expanded_bbox_contour.astype(int)], -1, (255, 0, 0), 2)
# Image.fromarray(img_drawn).show(title="Detected Transitions")

In [ ]:
# def get_arcs(paths):
#     """
#     Links the nodes of the paths based on the proximity_node attribute of the points.
#     This function assumes that the paths are already processed and contain points with proximity_node.
#     """
    
#     arcs = []

#     for path in paths:
#         if not path["points"][0].proximity_node or not path["points"][-1].proximity_node:
#             raise ValueError("Path must start and end with a proximity node.")
#         if len(path["points"]) < 2:
#             raise ValueError("Path must contain at least two points.")
#         if len(path["lines"]) < 1:
#             raise ValueError("Path must contain at least one line.")
#         if len(path["points"]) != len(path["lines"]) * 2:
#             raise ValueError("Path points and lines are inconsistent.")

#         start_point = path["points"][0]
#         end_point = path["points"][-1]

#         if end_point.is_arrow and not start_point.is_arrow:
#             arcs.append(Arc(
#                 source=start_point.proximity_node,
#                 target=end_point.proximity_node,
#                 start_point=start_point,
#                 end_point=end_point,
#                 points=path["points"],
#                 lines=path["lines"]
#             ))
#         if start_point.is_arrow and not end_point.is_arrow:
#             arcs.append(Arc(
#                 source=end_point.proximity_node,
#                 target=start_point.proximity_node,
#                 start_point=end_point,
#                 end_point=start_point,
#                 points=path["points"],
#                 lines=path["lines"]
#             ))
#         if not end_point.is_arrow and not start_point.is_arrow:
#             arcs.append(Arc(
#                 source=start_point.proximity_node,
#                 target=end_point.proximity_node,
#                 start_point=start_point,
#                 end_point=end_point,
#                 points=path["points"],
#                 lines=path["lines"]
#             ))
#         if start_point.is_arrow and end_point.is_arrow:
#             arcs.append(Arc(
#                 source=start_point.proximity_node,
#                 target=end_point.proximity_node,
#                 start_point=start_point,
#                 end_point=end_point,
#                 points=path["points"],
#                 lines=path["lines"]
#             ))
#             arcs.append(Arc(
#                 source=end_point.proximity_node,
#                 target=start_point.proximity_node,
#                 start_point=end_point,
#                 end_point=start_point,
#                 points=path["points"],
#                 lines=path["lines"]
#             ))

#     return arcs

# arcs = get_arcs(paths_with_arrows)
# arcs[:5]


In [ ]:
# def minmaxToWidthHeight(xyxy):
#     x1, y1, x2, y2 = xyxy
#     return (int(x1), int(y1), int(x2 - x1), int(y2 - y1))  # Convert to (x, y, width, height)

# def minmaxToCenterWidthHeight(xyxy):
#     x1, y1, x2, y2 = xyxy
#     center_x = int((x1 + x2) / 2)
#     center_y = int((y1 + y2) / 2)
#     width = int(x2 - x1)
#     height = int(y2 - y1)
#     return (center_x, center_y, width, height)  # Convert to (center_x, center_y, width, height)